In [1]:
#Python Implementation of the Hartree Fock Method
#Procedures listed in the code follow as described in Modern Quantum Chemistry: Introduction to Advanced Electronic Structure Theory, By Attila Szabo and Neil S. Ostlund

import math
import numpy as np
from molecule import atom
from molecule import vector
from molecule import guassian
from molecule import molecule
import integrals

#print(g3.constant * pow((math.pi/g3.orbitalExponet), 3/2))	

#Step 1
#Specify Molecules, Nuclear Coordinates, and Charge of the nucli Number of Electrons,

#generate an h2 atom with a distance of 1.4 AU to compare with Szabo pg. 175
h2 = molecule()
h2.addAtom(atom(vector(9,9,10.4), 1, 1))
h2.addAtom(atom(vector(9,9,9), 1, 1))
#h2.addAtom(atom(vector(5,0,0),1,1))
h2.addBasis("STO-3G")

#and a basis set, for now will add default basis of STO-3G for hydrogen to all atoms


In [2]:

#Step 2
#Calculate Integrals
#Overlap, KE, Nuclear Attraaction, Electronb 
S = integrals.overlap(h2)
print(np.matrix(S))

T = integrals.kineticEnergy(h2)
print()
print(np.matrix(T))
V = integrals.nuclearAttraction(h2)
V1 = integrals.electronNuclearAttraction(h2)
print()
print(np.matrix(V1))
print()
print(np.matrix(V[0]))
print()
print(np.matrix(V[1]))


[[0.99999999 0.6593182 ]
 [0.6593182  0.99999999]]

[[0.76003188 0.23645465]
 [0.23645465 0.76003188]]
GB Len = 3

X: 9 Y: 9 Z: 10.4

X: 9 Y: 9 Z: 10.4

X: 9 Y: 9 Z: 10.4
-------------------------------------
Atom: 0 Index1: 0 Index2: 0
Atom: 0 Index1: 0 Index2: 0
Atom: 0 Index1: 0 Index2: 0
Atom: 0 Index1: 0 Index2: 0
Atom: 0 Index1: 0 Index2: 0
Atom: 0 Index1: 0 Index2: 0
Atom: 0 Index1: 0 Index2: 0
Atom: 0 Index1: 0 Index2: 0
Atom: 0 Index1: 0 Index2: 0
Atom: 0 Index1: 0 Index2: 1
Atom: 0 Index1: 0 Index2: 1
Atom: 0 Index1: 0 Index2: 1
Atom: 0 Index1: 0 Index2: 1
Atom: 0 Index1: 0 Index2: 1
Atom: 0 Index1: 0 Index2: 1
Atom: 0 Index1: 0 Index2: 1
Atom: 0 Index1: 0 Index2: 1
Atom: 0 Index1: 0 Index2: 1
Atom: 0 Index1: 1 Index2: 0
Atom: 0 Index1: 1 Index2: 0
Atom: 0 Index1: 1 Index2: 0
Atom: 0 Index1: 1 Index2: 0
Atom: 0 Index1: 1 Index2: 0
Atom: 0 Index1: 1 Index2: 0
Atom: 0 Index1: 1 Index2: 0
Atom: 0 Index1: 1 Index2: 0
Atom: 0 Index1: 1 Index2: 0
Atom: 0 Index1: 1 Index2: 1
Atom: 0

In [3]:
#Step 3
#Dignolize Overlap Matrix and Obtain transformation matrix X

In [4]:
#Step 4
#Obtain guess Density matrix, P

In [5]:
#Step 5 
#Calculate Contraction of Density Matrix with Electron Repulsion, G Matrix

In [6]:
#Step 6
#Calculate Fock matrix, core matrix + G matrix

In [7]:
#Step 7
#Using X diagnolize Fock Matrix

In [8]:
#Step 8
#Diagnolize F' to get C' and E'

In [9]:
#Step 9
#Calculate C from C'

In [10]:
#Step 10
#From P for next iteration of HF Procedure

In [11]:
#Step 11
#Determine if current P is close "enough" to previos P, 
#	if no iterate again start from step 5
#	if yes, then stop iterating, HF procedure completed, and use final C matrix to compute physical properties
